<a href="https://colab.research.google.com/github/kasodeep/inheritance-project/blob/main/llama_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [38]:
!pip install langchain
!pip install fpdf
!pip install ctransformers
!pip install pypdf
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00


In [27]:
#imports
import langchain
from torch import cuda, bfloat16
from fpdf import FPDF
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader,PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain,ConversationalRetrievalChain,StuffDocumentsChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import CTransformers
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import PromptTemplate, LLMChain

In [28]:
#loading downloaded llm suitable for local usage,temperature(entropy/randomness in answer):1e-2
# But first you would need to mount your Google Drive to Google Colab.
llm = CTransformers(model=r"/content/drive/MyDrive/llama-2-13b.ggmlv3.q2_K.bin", model_type="llama", streaming=True,
                    callbacks=[StreamingStdOutCallbackHandler()],
                    config={'max_new_tokens':4096,'temperature':0.01, 'context_length':4096})

In [29]:
def load_documents(file_path):
    # Assuming you have a function to load documents (e.g., PyPDFLoader)
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    return docs

def summary_generation(file_path):
    # Load documents
    docs = load_documents(file_path)

    # Map
    map_template = """The following is a set of documents
    {docs}
    Based on this list of docs, please identify the main themes and concepts
    Expand the description of each topic and concept for 2-3 lines that should include its basic descriptions, key points, and formulas if any.
    Helpful Answer:"""
    map_prompt = PromptTemplate.from_template(map_template)
    map_chain = LLMChain(llm=llm, prompt=map_prompt)

    # Reduce
    reduce_template = """The following is a set of summaries:
    {docs}
    Take these and distill it into a final, consolidated summary of the main topics and concepts that should include definitions and formulas of the concepts. Mention all the key points and formulas related to a concept.
    Expand the description of each topic and concept for 2-3 lines.
    Helpful Answer:"""
    reduce_prompt = PromptTemplate.from_template(reduce_template)

    # Run chain
    reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

    # Takes a list of documents, combines them into a single string, and passes this to an LLMChain
    combine_documents_chain = StuffDocumentsChain(
        llm_chain=reduce_chain, document_variable_name="docs"
    )

    # Combines and iteratively reduces the mapped documents
    reduce_documents_chain = ReduceDocumentsChain(
        # This is the final chain that is called.
        combine_documents_chain=combine_documents_chain,
        # If documents exceed context for `StuffDocumentsChain`
        collapse_documents_chain=combine_documents_chain,
        # The maximum number of tokens to group documents into.
        token_max=4000,
    )

    # Combining documents by mapping a chain over them, then combining results
    map_reduce_chain = MapReduceDocumentsChain(
        # Map chain
        llm_chain=map_chain,
        # Reduce chain
        reduce_documents_chain=reduce_documents_chain,
        # The variable name in the llm_chain to put the documents in
        document_variable_name="docs",
        # Return the results of the map steps in the output
        return_intermediate_steps=False,
    )

    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=50
    )
    split_docs = text_splitter.split_documents(docs)
    all_summaries = map_reduce_chain.run(split_docs)
    print(all_summaries)

    return all_summaries



In [32]:
summary_generation(r"/content/transcript_output.pdf")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



        The following is a set of documents
            Language Code: en
Pakistani journalist aru kazmi is also now joining us thank you so much for your time uh by and large
the response by far of Pakistan is being seen as a Meek sort of a response statement being issued
that two children have been killed three women three girls have been injured also how do you
look into this response of Pakistan um I think Pakistan is not informing the right news uh as a
journalist I feel that because how this possible that Iran attack Pakistan to to the militant group and it
it kills two children only and three injured so there's something we are waiting for the full news and I
think the complete uh news we will get after few hours because it's too early that I comment on this
and uh but still foreign office issued this uh statement or uh s this is what I want to say that yes this
militant group was involved and attack Iran several times so that's why we are not surprised that Iran
tried to kill 

KeyboardInterrupt: 

-------------------------------------------------------------------------

In [ ]:
!pip install fpdf
!pip install pytube
!pip install youtube-transcript-api
!pip install fpdf
!pip install youtube_transcript_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 838.2 kB/s eta 0:00:00


In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from fpdf import *

def get_transcript(youtube_url, output_pdf_path):
    video_id = youtube_url.split("v=")[-1]
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

    # Try fetching the manual transcript
    try:
        transcript = transcript_list.find_manually_created_transcript()
        language_code = transcript.language_code  # Save the detected language
    except:
        # If no manual transcript is found, try fetching an auto-generated transcript in a supported language
        try:
            generated_transcripts = [trans for trans in transcript_list if trans.is_generated]
            transcript = generated_transcripts[0]
            language_code = transcript.language_code  # Save the detected language
        except:
            # If no auto-generated transcript is found, raise an exception
            raise Exception("No suitable transcript found.")

    full_transcript = " ".join([part['text'] for part in transcript.fetch()])

    # Save the transcript to a PDF file
    save_to_pdf(full_transcript, language_code, output_pdf_path)

    return full_transcript, language_code  # Return both the transcript and detected language

def save_to_pdf(transcript, language_code, output_pdf_path):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    pdf.cell(200, 10, txt=f"Language Code: {language_code}", ln=True, align='C')
    pdf.ln(10)

    pdf.multi_cell(0, 10, txt=transcript)

    pdf.output(output_pdf_path)

# Example usage
output_pdf_path = "transcript_output.pdf"
get_transcript("https://www.youtube.com/watch?v=tsJdXpnl48g&list=RDNStsJdXpnl48g&start_radio=1", output_pdf_path)

("Pakistani journalist aru kazmi is also now joining us thank you so much for your time uh by and large the response by far of Pakistan is being seen as a Meek sort of a response statement being issued that two children have been killed three women three three girls have been injured also how do you look into this response of Pakistan um I think Pakistan is not informing the right news uh as a journalist I feel that because how this possible that Iran attack Pakistan to to the militant group and it it kills two children only and three injured so there's something we are waiting for the full news and I think the complete uh news we will get after few hours because it's too early that I comment on this and uh but still foreign office issued this uh statement or uh s this is what I want to say that yes this militant group was involved and attack Iran several times so that's why we are not surprised that Iran tried to kill them or try to hit them but yes uh we are waiting for ispr and also

In [35]:
!pip install nltk gensim spacy

In [36]:
!python -m spacy download en_core_web_sm

2024-01-19 22:07:38.568079: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 22:07:38.568228: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 22:07:38.748253: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-19 22:07:45.015353: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 51.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [55]:
import PyPDF2
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

In [45]:
!pip install nltk

In [46]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    preprocessed_text = [lemmatizer.lemmatize(word.lower()) for word in word_tokens if word.isalnum() and word.lower() not in stop_words]
    return preprocessed_text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [47]:
from gensim import corpora, models

def perform_lda(preprocessed_text, num_topics=5):
    dictionary = corpora.Dictionary([preprocessed_text])
    corpus = [dictionary.doc2bow(preprocessed_text)]
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary)
    topics = lda_model.print_topics()
    return topics

In [56]:
pdf_path = r"/content/transcript_output.pdf"
text = extract_text_from_pdf(pdf_path)
preprocessed_text = preprocess_text(text)
topics = perform_lda(preprocessed_text)
print(topics)


[(0, '0.041*"uh" + 0.030*"pakistan" + 0.018*"think" + 0.016*"three" + 0.015*"response" + 0.014*"iran" + 0.013*"attack" + 0.012*"know" + 0.012*"journalist" + 0.012*"country"'), (1, '0.036*"uh" + 0.034*"pakistan" + 0.023*"think" + 0.017*"also" + 0.014*"iran" + 0.014*"time" + 0.013*"attack" + 0.013*"response" + 0.013*"three" + 0.011*"yes"'), (2, '0.027*"uh" + 0.023*"pakistan" + 0.018*"think" + 0.014*"three" + 0.013*"iran" + 0.011*"response" + 0.011*"attack" + 0.010*"also" + 0.010*"far" + 0.010*"time"'), (3, '0.058*"uh" + 0.052*"pakistan" + 0.022*"iran" + 0.018*"time" + 0.017*"also" + 0.016*"response" + 0.015*"attack" + 0.015*"think" + 0.015*"election" + 0.014*"three"'), (4, '0.042*"uh" + 0.035*"pakistan" + 0.016*"attack" + 0.015*"think" + 0.014*"three" + 0.013*"iran" + 0.012*"response" + 0.012*"time" + 0.012*"also" + 0.011*"yes"')]
